In [54]:
#Dependencies
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
import psycopg2

from config import protocol, username, password, host, port, database_name


In [55]:
#Extracting Files (Kdrama)
kdrama_file = "./Data/imdb.csv"
kdrama_df = pd.read_csv(kdrama_file)
kdrama_df.head()

,kdrama_name,imdb_rating,imdb_users,imdb_description
0,100 Days My Prince,7.7,"2,188","Upon losing his memory, a crown prince encount..."
1,12 Years Promise,7.3,181,A pregnant teen is forced by her family to lea...
2,18 Again,8.2,"1,847",A 37-year-old man on the verge of being divorc...
3,365: Repeat the Year,8.0,656,A story where ten people get the chance to go ...
4,7th Grade Civil Servant,6.3,145,A romantic comedy about a spy couple who hides...


In [56]:
#Cleaning Files (kdrama)
cleaned_kdrama_df = kdrama_df[["kdrama_name", "imdb_rating"]].copy()
cleaned_kdrama_df.sort_values(by=['imdb_rating'], inplace=True, ascending=False)
cleaned_kdrama_df.head()

,kdrama_name,imdb_rating
244,If You Wish Upon Me,9.5
452,Reply 1988,9.2
518,Standby,9.2
158,Extraordinary Attorney Woo,9.1
373,My Mister,9.1


In [57]:
##Extracting Files (Genres)
kdrama_genre_file = "./Data/genres.csv"
kdrama_genre_file_df = pd.read_csv(kdrama_genre_file)
kdrama_genre_file_df.head()

,kdrama_name,genre
0,100 Days My Prince,historical
1,100 Days My Prince,comedy
2,100 Days My Prince,romance
3,100 Days My Prince,drama
4,12 Signs of Love,comedy


In [58]:
#Merging dataframes 
kdrama_joined= pd.merge(cleaned_kdrama_df, kdrama_genre_file_df, on='kdrama_name', how='inner')
kdrama_joined.head()

,kdrama_name,imdb_rating,genre
0,If You Wish Upon Me,9.5,life
1,If You Wish Upon Me,9.5,drama
2,If You Wish Upon Me,9.5,medical
3,Reply 1988,9.2,comedy
4,Reply 1988,9.2,romance


In [59]:
#Capitalizing Genre
kdrama_joined["genre"] = kdrama_joined["genre"].str.upper()
kdrama_joined.head()

,kdrama_name,imdb_rating,genre
0,If You Wish Upon Me,9.5,LIFE
1,If You Wish Upon Me,9.5,DRAMA
2,If You Wish Upon Me,9.5,MEDICAL
3,Reply 1988,9.2,COMEDY
4,Reply 1988,9.2,ROMANCE


In [60]:
#Appending Genres into One Row
kdrama_joined_genre_df = kdrama_joined.groupby(['kdrama_name'])['genre'].apply(lambda x : np.unique(list(x))).reset_index()


In [61]:
#Recreating IMDB Rating Dataframe 
kdrama_joined = kdrama_joined[["kdrama_name", "imdb_rating"]]
kdrama_joined.head()


,kdrama_name,imdb_rating
0,If You Wish Upon Me,9.5
1,If You Wish Upon Me,9.5
2,If You Wish Upon Me,9.5
3,Reply 1988,9.2
4,Reply 1988,9.2


In [62]:
#Combining Rating and Genre Dataframes
ratings_genre_combined = pd.merge(kdrama_joined, kdrama_joined_genre_df, on='kdrama_name', how='inner')
final_df = ratings_genre_combined.drop_duplicates(subset="kdrama_name")
final_df.head()

,kdrama_name,imdb_rating,genre
0,If You Wish Upon Me,9.5,"[ DRAMA, MEDICAL, LIFE]"
3,Reply 1988,9.2,"[ LIFE, ROMANCE, YOUTH, COMEDY]"
7,Standby,9.2,"[ COMEDY, SITCOM, BUSINESS]"
10,Extraordinary Attorney Woo,9.1,"[ DRAMA, LIFE, ROMANCE, LAW]"
14,My Mister,9.1,"[ DRAMA, LIFE, PSYCHOLOGICAL]"


In [63]:
#Removing all IMDB Rating < 8 
final_df = final_df[final_df['imdb_rating'] >= 8] 
final_df.head()

,kdrama_name,imdb_rating,genre
0,If You Wish Upon Me,9.5,"[ DRAMA, MEDICAL, LIFE]"
3,Reply 1988,9.2,"[ LIFE, ROMANCE, YOUTH, COMEDY]"
7,Standby,9.2,"[ COMEDY, SITCOM, BUSINESS]"
10,Extraordinary Attorney Woo,9.1,"[ DRAMA, LIFE, ROMANCE, LAW]"
14,My Mister,9.1,"[ DRAMA, LIFE, PSYCHOLOGICAL]"


In [64]:
#Titlecasing Kdrama Name
final_df["kdrama_name"] = final_df["kdrama_name"].str.title()
final_df.head()

,kdrama_name,imdb_rating,genre
0,If You Wish Upon Me,9.5,"[ DRAMA, MEDICAL, LIFE]"
3,Reply 1988,9.2,"[ LIFE, ROMANCE, YOUTH, COMEDY]"
7,Standby,9.2,"[ COMEDY, SITCOM, BUSINESS]"
10,Extraordinary Attorney Woo,9.1,"[ DRAMA, LIFE, ROMANCE, LAW]"
14,My Mister,9.1,"[ DRAMA, LIFE, PSYCHOLOGICAL]"


In [65]:
#Resetting Dataframe Index
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,kdrama_name,imdb_rating,genre
0,If You Wish Upon Me,9.5,"[ DRAMA, MEDICAL, LIFE]"
1,Reply 1988,9.2,"[ LIFE, ROMANCE, YOUTH, COMEDY]"
2,Standby,9.2,"[ COMEDY, SITCOM, BUSINESS]"
3,Extraordinary Attorney Woo,9.1,"[ DRAMA, LIFE, ROMANCE, LAW]"
4,My Mister,9.1,"[ DRAMA, LIFE, PSYCHOLOGICAL]"


In [66]:
connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(connection_string)

In [67]:
engine.table_names()

/var/folders/06/dlzbbxfn6tgfd6s50hkzfk3r0000gn/T/ipykernel_79515/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['kdrama']

In [68]:
final_df.to_sql(name='kdrama', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'numpy.ndarray'
[SQL: INSERT INTO kdrama (kdrama_name, imdb_rating, genre) VALUES (%(kdrama_name)s, %(imdb_rating)s, %(genre)s)]
[parameters: ({'kdrama_name': 'If You Wish Upon Me', 'imdb_rating': 9.5, 'genre': array([' DRAMA', ' MEDICAL', 'LIFE'], dtype='<U8')}, {'kdrama_name': 'Reply 1988', 'imdb_rating': 9.2, 'genre': array([' LIFE', ' ROMANCE', ' YOUTH', 'COMEDY'], dtype='<U8')}, {'kdrama_name': 'Standby', 'imdb_rating': 9.2, 'genre': array([' COMEDY', ' SITCOM', 'BUSINESS'], dtype='<U8')}, {'kdrama_name': 'Extraordinary Attorney Woo', 'imdb_rating': 9.1, 'genre': array([' DRAMA', ' LIFE', ' ROMANCE', 'LAW'], dtype='<U8')}, {'kdrama_name': 'My Mister', 'imdb_rating': 9.1, 'genre': array([' DRAMA', ' LIFE', 'PSYCHOLOGICAL'], dtype='<U13')}, {'kdrama_name': 'The Stars Are Shining', 'imdb_rating': 9.1, 'genre': array([' DRAMA', ' FAMILY', ' MELODRAMA', ' ROMANCE', 'HISTORICAL'],
      dtype='<U10')}, {'kdrama_name': 'Sweet